In [1]:
include(dirname(dirname(pwd()))*"\\src\\TuLiPa.jl");
using .TuLiPa
using Dates, DataFrames, CSV, JSON, Plots, JuMP, HiGHS
include(dirname(dirname(pwd())) * raw"\demos\other\flowbased_utils.jl")

remove_cap_for_flowbased_transm (generic function with 1 method)

In [2]:
PowerA = 1.0
PowerA_cap = 20.0
DemandA = 10.0
PowerB_cap = 1000000000.0
PowerB = 100000000.0
DemandB = 100.0
PowerC_cap = 1000000000.0 
PowerC = 100.0
DemandC = 10.0


A = "A"
B = "B"
C = "C"

function make_connection(from, to, prefix = "")  
    
    transm_from = replace(from, ("PowerBalance_" => ""))
    transm_to = replace(to, ("PowerBalance_" => ""))
    
    flow_name = "Transm_$(transm_from)->$(transm_to)" * prefix
    arrow_name_from = "arrow_from$(from)$(to)" * prefix
    arrow_name_to = "arrow_to$(from)$(to)" * prefix
    balance_from = "$(from)"
    balance_to = "$(to)"
    cap_name = "$(flow_name)_cap"
    elem = [
        DataElement("Flow", "BaseFlow", flow_name, Dict()),    
        DataElement("Arrow", "BaseArrow", arrow_name_from, Dict("Balance" => balance_from, "Flow" => flow_name, "Conversion" => 1.0, "Direction" => "Out")),
        DataElement("Arrow", "BaseArrow", arrow_name_to, Dict("Balance" => balance_to, "Flow" => flow_name, "Conversion" => 1.0, "Direction" => "In")),
    ]    
    return elem
end

make_connection (generic function with 2 methods)

In [3]:
create_duplicated_flow = false

false

In [4]:
function create_elements()
    elements = [

        DataElement("Balance", "BaseBalance", A, Dict{Any, Any}("Commodity" => "Power")),    
        DataElement("Flow", "BaseFlow", "PowerA", Dict{Any, Any}()),
        DataElement("Arrow", "BaseArrow", "ArrowA", Dict{Any, Any}("Balance" => A, "Flow" => "PowerA", "Conversion" => 1.0, "Direction" => "In")),
        DataElement("Cost", "CostTerm", "PowerA_cost", Dict{Any, Any}("Param" => PowerA, "WhichInstance" => "PowerA", "WhichConcept" => "Flow", "Direction" => "In")),
        DataElement("Capacity", "PositiveCapacity", "PowerA_cap", Dict{Any, Any}("Param" => "PowerA_cap", "WhichInstance" => "PowerA", "WhichConcept" => "Flow", "Bound" => "Upper")),
        DataElement("Param", "MWToGWhSeriesParam", "PowerA_cap", Dict{Any, Any}("Level" => PowerA_cap, "Profile" => 1.0)),
        DataElement("Param", "MWToGWhSeriesParam", "DemandA", Dict("Level" => DemandA , "Profile" => 1.0)),
        DataElement("RHSTerm", "BaseRHSTerm", "DemandA", Dict{Any, Any}("Balance" => A, "Param" => "DemandA", "Direction" => "Out")),

        DataElement("Balance", "BaseBalance", B, Dict{Any, Any}("Commodity" => "Power")),    
        DataElement("Flow", "BaseFlow", "PowerB", Dict{Any, Any}()),
        DataElement("Arrow", "BaseArrow", "ArrowB", Dict{Any, Any}("Balance" => B, "Flow" => "PowerB", "Conversion" => 1.0, "Direction" => "In")),
        DataElement("Cost", "CostTerm", "PowerB_cost", Dict{Any, Any}("Param" => PowerB, "WhichInstance" => "PowerB", "WhichConcept" => "Flow", "Direction" => "In")),
        DataElement("Capacity", "PositiveCapacity", "PowerB_cap", Dict{Any, Any}("Param" => "PowerB_cap", "WhichInstance" => "PowerB", "WhichConcept" => "Flow", "Bound" => "Upper")),
        DataElement("Param", "MWToGWhSeriesParam", "PowerB_cap", Dict{Any, Any}("Level" => PowerB_cap, "Profile" => 1.0)),
        DataElement("Param", "MWToGWhSeriesParam", "DemandB", Dict("Level" => DemandB , "Profile" => 1.0)),
        DataElement("RHSTerm", "BaseRHSTerm", "DemandB", Dict{Any, Any}("Balance" => B, "Param" => "DemandB", "Direction" => "Out")),

        DataElement("Balance", "BaseBalance", C, Dict{Any, Any}("Commodity" => "Power")),    
        DataElement("Flow", "BaseFlow", "PowerC", Dict{Any, Any}()),
        DataElement("Arrow", "BaseArrow", "ArrowC", Dict{Any, Any}("Balance" => C, "Flow" => "PowerC", "Conversion" => 1.0, "Direction" => "In")),
        DataElement("Cost", "CostTerm", "PowerC_cost", Dict{Any, Any}("Param" => PowerC, "WhichInstance" => "PowerC", "WhichConcept" => "Flow", "Direction" => "In")),
        DataElement("Capacity", "PositiveCapacity", "PowerC_cap", Dict{Any, Any}("Param" => "PowerC_cap", "WhichInstance" => "PowerC", "WhichConcept" => "Flow", "Bound" => "Upper")),
        DataElement("Param", "MWToGWhSeriesParam", "PowerC_cap", Dict{Any, Any}("Level" => PowerC_cap, "Profile" => 1.0)),
        DataElement("Param", "MWToGWhSeriesParam", "DemandC", Dict("Level" => DemandC , "Profile" => 1.0)),
        DataElement("RHSTerm", "BaseRHSTerm", "DemandC", Dict{Any, Any}("Balance" => C, "Param" => "DemandC", "Direction" => "Out"))
    ]

    elements = vcat(elements, make_connection(A, B))
    elements = vcat(elements, make_connection(B, A))

    elements = vcat(elements, make_connection(A, C))
    elements = vcat(elements, make_connection(C, A))
    
    if create_duplicated_flow
        elements = vcat(elements, make_connection(A, C, "|1"))
        elements = vcat(elements, make_connection(C, A, "|1"))  
    end
            
    elements = vcat(elements, make_connection(B, C))
    elements = vcat(elements, make_connection(C, B))

    power_horizon = SequentialHorizon(1, Day(1))
    push!(elements, getelement(COMMODITY_CONCEPT, "BaseCommodity", "Power", (HORIZON_CONCEPT, power_horizon)))
    addscenariotimeperiod!(elements, "ScenarioTimePeriod", getisoyearstart(1981), getisoyearstart(1983));
    return elements
end

create_elements (generic function with 1 method)

In [7]:
df = DataFrame(
    line=["Transm_A->B", "Transm_A->C", "Transm_A->C|1",  "Transm_B->C"],
    RAM = [100000.0, 0.2, 400, 100000.0],
    A = [0.2, 0.4, 0.4, 0.2],
    B = [-0.4, 0.2, 0.2, 0.6],
    C = [0, 0, 0, 0]
)

function process_df_aggregate(df)
    df_copy = copy(df)
    df_copy.RAM .= missing
    df_copy.border .= true
    df_copy = df_copy[df_copy.line .!= "Transm_A->C|1", :]
    df_copy[2, Not(["line", "RAM", "border"])] = [0.8, 0.4, 0]

    df.border .= false
    df.line = "non_flow" .* df.line
    df = vcat(df, df_copy)
    df = select(df, ["line", "RAM", "border"], Not(["line", "RAM", "border"]))
    return df
end

function process_df(df)
    df_copy = copy(df)
    df_copy.RAM .= missing
    df_copy.border .= true

    df.border .= false
    df.line = "non_flow" .* df.line
    df = vcat(df, df_copy)
    df = select(df, ["line", "RAM", "border"], Not(["line", "RAM", "border"]))
    return df
end

if create_duplicated_flow
    df = process_df(df)
else
    df = process_df_aggregate(df)
end

Row,line,RAM,border,A,B,C
,String,Float64?,Bool,Float64,Float64,Int64
1,non_flowTransm_A->B,100000.0,false,0.2,-0.4,0
2,non_flowTransm_A->C,0.2,false,0.4,0.2,0
3,non_flowTransm_A->C|1,400.0,false,0.4,0.2,0
4,non_flowTransm_B->C,100000.0,false,0.2,0.6,0
5,Transm_A->B,missing,true,0.2,-0.4,0
6,Transm_A->C,missing,true,0.8,0.4,0
7,Transm_B->C,missing,true,0.2,0.6,0


In [6]:
elements = create_elements()
transm = [e for e in elements if is_transmission(e)]
T = DataElement
flow_based = make_flowbased(df, transm)
elements = vcat(elements,  flow_based)

modelobjects = getmodelobjects(elements)
mymodel = JuMP.Model(HiGHS.Optimizer)
prob = JuMP_Prob(modelobjects, mymodel)
datatime = getisoyearstart(2023)
scenariotime = getisoyearstart(1981)
prob_time = TwoTime(datatime, scenariotime)
update!(prob, prob_time)
solve!(prob)
print(prob.model)
print(solution_summary(prob.model, verbose = true))

* Solver : HiGHS

* Status
  Result count       : 1
  Termination status : OPTIMAL
  Message from the solver:
  "kHighsModelStatusOptimal"

* Candidate solution (result #1)
  Primal status      : FEASIBLE_POINT
  Dual status        : FEASIBLE_POINT
  Objective value    : 9.20001e+07
  Objective bound    : 0.00000e+00
  Relative gap       : Inf
  Dual objective value : 9.20001e+07
  Primal solution :
    FlowPowerA[1] : 4.80000e-01
    FlowPowerB[1] : 9.20000e-01
    FlowPowerC[1] : 1.48000e+00
    FlowTransm_A->B[1] : 6.40000e-01
    FlowTransm_A->C[1] : 0.00000e+00
    FlowTransm_B->A[1] : 0.00000e+00
    FlowTransm_B->C[1] : 0.00000e+00
    FlowTransm_C->A[1] : 4.00000e-01
    FlowTransm_C->B[1] : 8.40000e-01
  Dual solution :
    BalanceA[1] : -2.00000e+08
    BalanceB[1] : -1.00000e+08
    BalanceC[1] : -1.00000e+02
    FlowBasedTransm_A->B[1] : -0.00000e+00
    FlowBasedTransm_A->C[1] : -0.00000e+00
    FlowBasedTransm_B->C[1] : -0.00000e+00
    FlowBasednon_flowTransm_A->Bin[1] :